In [1]:
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p polars,instructor,openai,ollama

Author: Prayson W. Daniel

Last updated: 2024-10-31T09:11:19.045838+01:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

polars    : 1.7.1
instructor: 1.4.3
openai    : 1.46.1
ollama    : 0.3.3

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
from typing import Literal, Final
from openai import OpenAI
from pydantic import BaseModel, Field
import polars as pl
import instructor

In [3]:
MODEL: Final = "gemma2"

VALID_EMOTIONS: tuple[str, ...] = tuple(
    {
        "admiration",
        "approval",
        "gratitude",
        "optimism",
        "joy",
        "neutral",
        "disapproval",
        "disappointment",
        "love",
        "relief",
        "excitement",
        "desire",
        "caring",
        "pride",
        "realization",
        "disgust",
        "annoyance",
        "remorse",
        "amusement",
        "anger",
        "grief",
        "curiosity",
        "fear",
        "surprise",
        "sadness",
        "embarrassment",
        "confusion",
        "nervousness",
    }
)

In [4]:
class Emotion(BaseModel):
    kind: Literal[VALID_EMOTIONS]  # type: ignore # literal does not like dynamic values
    reason: str = Field(
        ..., description="A short third person reason for selected the emotion"
    )

In [5]:
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

In [6]:
def classifier(text: str, model: str = MODEL) -> Emotion:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": f"You are a brilliant emotion pyschologist. What is the text emotion: Text: {text}",
            }
        ],
        response_model=Emotion,
        temperature=0.0,
    )

    return response

In [7]:
pl.Config.set_fmt_str_lengths(100).set_tbl_rows(
    -1
)  # show more text. default is 25, show all rowss

dataf = pl.read_json("../data/examples.json").explode(["emotion", "text"])

In [8]:
dataf.head(3)

emotion,text
str,str
"""admiration""","""The way she solved that complex math problem was incredible. Truly a mind to admire!"""
"""admiration""","""He ran a marathon in record time despite his injury. What a remarkable feat!"""
"""admiration""","""Her dedication to helping the homeless inspires me every day."""


In [9]:
classifier("I am sad!")

Emotion(kind='sadness', reason='The person explicitly states they are sad.')

In [10]:
(
    dataf.sample(15)
    .with_columns(
        pl.col("text")
        .map_elements(classifier, return_dtype=pl.Object)
        .alias("prediction")
    )
    .with_columns(
        pl.col("prediction")
        .map_elements(lambda p: p.kind, return_dtype=pl.String)
        .alias("prediction"),
        pl.col("prediction")
        .map_elements(lambda p: p.reason, return_dtype=pl.String)
        .alias("reason"),
    )
)

emotion,text,prediction,reason
str,str,str,str
"""disappointment""","""I was expecting a better outcome after all the effort we put in.""","""disappointment""","""The speaker expected a better result than what they achieved despite their efforts."""
"""gratitude""","""I really appreciate you covering my shift last minute. You're a lifesaver!""","""gratitude""","""The speaker expresses thanks for someone covering their shift, indicating they feel grateful."""
"""excitement""","""I can’t wait to see what this year has in store for us!""","""excitement""","""The speaker expresses anticipation and eagerness for the future."""
"""disgust""","""I feel disgusted by the lack of cleanliness at the venue.""","""disgust""","""the lack of cleanliness at the venue"""
"""disapproval""","""This proposal is filled with inconsistencies. It needs to be reworked.""","""disapproval""","""The proposal contains inconsistencies that require revision."""
"""sadness""","""It’s hard to shake this sadness after hearing such devastating news.""","""sadness""","""Hearing devastating news."""
"""surprise""","""His sudden resignation came as a complete surprise to everyone.""","""surprise""","""His unexpected decision to resign was unforeseen by all."""
"""disgust""","""That was a truly disgusting attitude displayed during the meeting.""","""disgust""","""The attitude displayed during the meeting was considered repulsive."""
"""sadness""","""I’ve felt a lingering sadness since the end of that relationship.""","""sadness""","""The end of a relationship has caused lingering feelings of sadness."""
